<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#UHS1–4-EA+AA-2df-Meta-Analysis-for-HIV-Acquisition" data-toc-modified-id="UHS1–4-EA+AA-2df-Meta-Analysis-for-HIV-Acquisition-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>UHS1–4 EA+AA 2df Meta-Analysis for HIV Acquisition</a></span></li><li><span><a href="#Download-Data-&amp;-Create-Directory-Structure" data-toc-modified-id="Download-Data-&amp;-Create-Directory-Structure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Download Data &amp; Create Directory Structure</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span><ul class="toc-item"><li><span><a href="#cross-ancestry" data-toc-modified-id="cross-ancestry-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>cross-ancestry</a></span></li></ul></li><li><span><a href="#Code" data-toc-modified-id="Code-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="#methods" data-toc-modified-id="methods-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>methods</a></span></li><li><span><a href="#Secondary-scripts" data-toc-modified-id="Secondary-scripts-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Secondary scripts</a></span><ul class="toc-item"><li><span><a href="#merge_metal" data-toc-modified-id="merge_metal-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>merge_metal</a></span></li><li><span><a href="#qsub_metal.sh" data-toc-modified-id="qsub_metal.sh-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>qsub_metal.sh</a></span></li></ul></li></ul></li></ul></div>

# UHS1–4 EA+AA 2df Meta-Analysis for HIV Acquisition
**Author:** Jesse Marks <br>
**Results**: The results of this meta-analysis will be stored on AWS S3 at: `s3:///rti-hiv/meta_new/020`<br>
**GitHub Issue:** [#97](https://github.com/RTIInternational/bioinformatics/issues/97)

# Download Data & Create Directory Structure

In [ ]:
baseD=/shared/jmarks/hiv/meta

## Download data from S3

## Create directory structure then mv data over to respective directories
mkdir -p $baseD/data/uhs1234/{1df,2df}/{ea,aa}
mkdir -p $baseD/results/uhs1234/{1df,2df}/001/{processing,final}

## unzip data

# Results

## cross-ancestry

In [ ]:
library("IRdisplay")
display_png(file="C://Users/Jess/OneDrive - Research Triangle Institute/Projects/HIV/uhs1-4/meta/002/uhs1234_hiv_acquisition_1df_meta_analysis_afr+eur.1df.snps+indels.manhattan.png", width=800, height=800)
display_png(file="C://Users/Jess/OneDrive - Research Triangle Institute/Projects/HIV/uhs1-4/meta/002/uhs1234_hiv_acquisition_1df_meta_analysis_afr+eur.1df.snps+indels.qq.png", width=800, height=800)

# Code
Below are the methods to perform the meta-analysis in the AWS environment. The main parts that need modified are the paths to the cohorts' data which has been modified. For example, the R section at the end on lines 315 and 316.

Also note that we are converting everything to the 1000 Genome Phase 3 format and therefore we will include the script for performing this below.

## methods
Perform meta-analysis with METAL and produce the Manhattan and QQ plots.

In [ ]:
df="2df" # degrees of freedom (1df or 2df)
study="uhs1-4_aa+uhs1-4_ea" # prepended to results files and for directory structure
study_num=020
filePrefix="hiv_acquisition_${df}_meta_analysis_${study}"
outDir="/shared/jmarks/hiv/meta/results/${study_num}"
dataDir="/shared/jmarks/hiv/meta/data"
minchr=1 # first chr in sequence
maxchr=2  # 22 or 23 

interactive_beta="beta_SNP_gender"
interactive_std="sebeta_SNP_gender"
interactive_cov="cov_SNP_int_SNP_gender"
mkdir -p $outDir/{processing,final}


# functions to create cohort paths. arg1="base/directory/path" arg2="chromosome#"
uhs1234_aa() { echo $1/uhs1234/$df/aa/uhs1234.aa.$df.1000G_p3.chr$2.HIV_ACQ~SNP+AGE+SEX+SNPbySEX+PC10+PC9+PC2+PC6.maf_gt_0.01_subject+afr.rsq_gt_0.30.stats ;}
uhs1234_ea() { echo $1/uhs1234/$df/ea/uhs1234.ea.$df.1000G_p3.chr$2.HIV_ACQ~SNP+AGE+SEX+SNPbySEX+PC1+PC9+PC10.maf_gt_0.01_subject+eur.rsq_gt_0.30.stats ;}
#vidus_ea() { echo $1/vidus/$df/ea/ ;}


#NOTE, the following loop only works running in a new vim shell without any
# predefined function. You could do this manually if you otherwise wanted to.
# create capitalized cohort labels based off of the above functions
cohortLabels=""
for function in $(compgen -A function);do
    cohortLabels+=$(echo ${function^^})
    cohortLabels+=" "
done

# remove "MODULE" function
echo $cohortLabels
cohortLabels=${cohortLabels//"MODULE"}
echo $cohortLabels

# the cohort labels below were retrieved from running the loop above
cohortLabels="UHS1234_AA UHS1234_EA"
################################################################################
################################################################################

##name    chrom   position        A1      A2      Freq1   MAF     Quality Rsq     n       Mean_predictor_allele   beta_SNP_addA1  sebeta_SNP_addA1      chi2_SNP_add     chi     p       or_95_percent_ci ##

if [ $df == "1df" ];then
    pv="p"
    scheme="STDERR"
elif [ $df == "2df" ]; then
    pv="p_2df"
    scheme="INTERACTION"
fi

### START METAL analysis ###
for (( chr=${minchr}; chr<=${maxchr}; chr++)); do
    my_string=""
    for cohort in ${cohortLabels};do
        string_fun() { echo "--PROCESS" $(${cohort,,} $1 $2); }
        my_string+=$(string_fun $dataDir $chr)
        my_string+=" "
    done
    bash ~/bin/qsub_metal.sh \
        --SCHEME $scheme \
        --PVALUE $pv \
        --MARKER name \
        --ALLELE A1 A2 \
        --EFFECT beta_SNP_addA1 \
        --STDERR sebeta_SNP_addA1 \
        --INTEFFECTLABEL $interactive_beta \
        --INTSTDERRLABEL $interactive_std \
        --INTCOVLABEL $interactive_cov \
        --GENOMICCONTROL OFF \
        ${my_string} \
        --OUTFILE $outDir/processing/${filePrefix}.chr$chr.$df .metal \
        --ANALYZE HETEROGENEITY \
        --script_prefix $outDir/processing/${filePrefix}.chr$chr.$df \
        --mem 3.8 \
        --priority 0
done

## START Merge ProbABEL and METAL results ###
for (( chr=${minchr}; chr<=${maxchr}; chr++ )); do
    my_string=""
    for cohort in ${cohortLabels};do
        string_fun() { echo $(${cohort,,} $1 $2); }
        my_string+=$(string_fun $dataDir $chr)
        my_string+=" "
    done

num_cohorts="$(echo $cohortLabels  | wc -w)"
chrom=$(printf '%0.s chrom' $(seq 1 ${num_cohorts}))
position=$(printf '%0.s position' $(seq 1 ${num_cohorts}))
name=$(printf '%0.s name' $(seq 1 ${num_cohorts}))
#name="$(for ((i=1; i<=${num_cohorts}; i++ )); do printf '%0s name'; done)"

  /shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name merge_meta_${chr} \
    --script_prefix $outDir/processing/${filePrefix}.chr$chr.$df \
    --mem 10 \
    --nslots 2 \
    --priority 0 \
    --program Rscript /shared/bioinformatics/software/R/merge_probabel_metal.R \
        --metal $outDir/processing/${filePrefix}.chr$chr.$df.metal \
        --chr_columns  $chrom \
        --pos_columns  $position \
        --probabel_merge_bys $name \
        --probabel \
        ${my_string} \
        --probabel_labels ${cohortLabels} \
        --out $outDir/final/${filePrefix}.chr$chr.$df 
        ${my_string} \
        --probabel_labels ${cohortLabels} \
        --out $outDir/final/${filePrefix}.chr$chr.$df 
done
### END Merge ProbABEL and METAL results ###

# Check for completion (should return/print out nothing)
for (( chr=${minchr}; chr<=${maxchr}; chr++ )); do
  file=$outDir/processing/${filePrefix}.chr$chr.$df.qsub.log
  if [ -f $file ]
  then
    logLineCount=$(wc -l $file | perl -lane 'print $F[0];')
    if [ $logLineCount -eq 0 ]; then
      echo $file empty
    else
      tail -n 1 $file |
        perl -ne 'chomp; if (!/Done$/) { print "'$file'\n".$_."\n"; }'
    fi
  else
    echo $file missing
  fi
done


### START Generate plots ###
# create SNP table for input to plotting script
if [ $df == "1df" ]; then
    skip_col=2
else
	skip_col=6
fi


outFile=$outDir/processing/${filePrefix}.$df.table
echo -e "VARIANT_ID\tCHR\tPOSITION\tP\tTYPE" > $outFile
for (( chr=${minchr}; chr<=${maxchr}; chr++ )); do
    inFile=$outDir/final/${filePrefix}.chr$chr.$df  
    echo Processing $inFile
    tail -n +2 $inFile |
    perl -slne '/^(\S+)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)(?:\s+\S+){$skip_col}\s+(\S+)/;
                if (($4 eq "a" || $4 eq "c" || $4 eq "g" || $4 eq "t") && ($5 eq "a" || $5 eq "c" || $5 eq "g" || $5 eq "t")) {
                  print join("\t",$1,$2,$3,$6,"snp");
                } else {
                  print join("\t",$1,$2,$3,$6,"indel");
                }' -- -skip_col=${skip_col} >> $outFile
done


# generate plots
/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name gwas_plots \
    --script_prefix $outDir/processing/${filePrefix}.$df.plots \
    --mem 10 \
    --nslots 3 \
    --priority 0 \
    --program Rscript /shared/bioinformatics/software/R/generate_gwas_plots.R \
    --in $outDir/processing/${filePrefix}.$df.table \
    --in_chromosomes autosomal_nonPAR \
    --in_header \
    --out $outDir/processing/${filePrefix}.$df\
    --col_id VARIANT_ID \
    --col_chromosome CHR \
    --col_position POSITION \
    --col_p P \
    --col_variant_type TYPE \
    --generate_snp_indel_manhattan_plot \
    --manhattan_odd_chr_color red \
    --manhattan_even_chr_color blue \
    --manhattan_points_cex 1.5 \
    --manhattan_cex_lab 1.5 \
    --manhattan_cex_axis 1.5 \
    --generate_snp_indel_qq_plot \
    --qq_lines \
    --qq_points_bg black \
    --qq_lambda

mv $outDir/processing/*.png \
 $outDir/final
### END Generate plots ###

                  
## remove singleton SNP/indels (snps founnd in only one cohort) ###
# python script
python 

#####################################################################
minchr = 1 # starting chr
maxchr = 24 # ending chr plus 1
study = "uhs1-4_aa+uhs1-4_ea" 
df = "2df"
filePrefix = "hiv_acquisition_{}_meta_analysis_{}.chr".format(df, study)
outDir = "/shared/jmarks/hiv/meta/results/020/final/" 

#####################################################################

if df == "1df":
    direction_col = 8
elif df == "2df":
    direction_col = 12

for chrom in range(minchr,maxchr):
    print("Processing chromosome " + str(chrom) + "...")
    inF = open(outDir + filePrefix + str(chrom) + "." + df, "r")
    outF = open(outDir + filePrefix + str(chrom) + ".exclude_singletons." + df, "w")
    firstLine = inF.readline()
    outF.write(firstLine)
    line = inF.readline()
    while(line):
        tmp = line.split()
        if(sum([d != "?" for d in tmp[direction_col]]) > 1):
            outF.write(line)
        line = inF.readline()
    inF.close()
    outF.close()
## end python script


### START removed SNPs with p_2df==0
python
#####################################################################
minchr = 1 # starting chr
maxchr = 24 # ending chr plus 1
study = "uhs1-4_aa+uhs1-4_ea"
df = "2df"
filePrefix = "hiv_acquisition_{}_meta_analysis_{}.chr".format(df, study)
outDir = "/shared/jmarks/hiv/meta/results/020/final/"

#####################################################################



for chrom in range(minchr,maxchr):
    print("Processing chromosome " + str(chrom) + "...")
    inF = open(outDir + filePrefix + str(chrom) + ".exclude_singletons." + df, "r")
    outF = open(outDir + filePrefix + str(chrom) + ".exclude_singletons+p2df_eq_0." + df, "w")
    firstLine = inF.readline()
    print(firstLine)
    sl = firstLine.split()
    p2df_list = []
    for item in sl:
        if "p_2df." in item:
            p2df_list.append(sl.index(item))

    outF.write(firstLine)
    line = inF.readline()
    while(line):
        zero = False
        tmp = line.split()
        for index in p2df_list:
            if tmp[index] == "0":
                print(line)
                zero = True
                break
        if zero == False:
            outF.write(line)

        line = inF.readline()
    inF.close()
    outF.close()
### end python script


### START Generate plots with singletons removed and SNPs with p_2df==0###
# create SNP table for input to plotting script
outFile=$outDir/processing/${filePrefix}.exclude_singletons+p2df_eq_0.$df.table
echo -e "VARIANT_ID\tCHR\tPOSITION\tP\tTYPE" > $outFile
for (( chr=${minchr}; chr<=${maxchr}; chr++ )); do
    inFile=$outDir/final/${filePrefix}.chr$chr.exclude_singletons+p2df_eq_0.$df  
    echo Processing $inFile
    tail -n +2 $inFile |
    perl -slne '/^(\S+)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)(?:\s+\S+){$skip_col}\s+(\S+)/;
                if (($4 eq "a" || $4 eq "c" || $4 eq "g" || $4 eq "t") && ($5 eq "a" || $5 eq "c" || $5 eq "g" || $5 eq "t")) {
                  print join("\t",$1,$2,$3,$6,"snp");
                } else {
                  print join("\t",$1,$2,$3,$6,"indel");
                }' -- -skip_col=${skip_col} >> $outFile
done


# Generate plots with singletons removed
/shared/bioinformatics/software/scripts/qsub_job.sh \
  --job_name gwas_plots_no_singletons \
  --script_prefix $outDir/processing/${filePrefix}.exclude_singletons+p2df_eq_0.$df.plots \
  --mem 10 \
  --nslots 3 \
  --priority 0 \
  --program Rscript /shared/bioinformatics/software/R/generate_gwas_plots.R \
  --in $outDir/processing/${filePrefix}.exclude_singletons+p2df_eq_0.$df.table \
  --in_chromosomes autosomal_nonPAR \
  --in_header \
  --out $outDir/processing/${filePrefix}.exclude_singletons+p2df_eq_0.$df \
  --col_id VARIANT_ID \
  --col_chromosome CHR \
  --col_position POSITION \
  --col_p P \
  --col_variant_type TYPE \
  --generate_snp_indel_manhattan_plot \
  --manhattan_odd_chr_color red \
  --manhattan_even_chr_color blue \
  --manhattan_points_cex 1.5 \
  --manhattan_cex_lab 1.5 \
  --manhattan_cex_axis 1.5 \
  --generate_snp_indel_qq_plot \
  --qq_lines \
  --qq_points_bg black \
  --qq_lambda


# plot with ylim
/shared/bioinformatics/software/scripts/qsub_job.sh \
  --job_name gwas_plots_no_singletons_ylim \
  --script_prefix $outDir/processing/${filePrefix}.exclude_singletons+p2df_eq_0.$df.ylim_9.plots \
  --mem 10 \
  --nslots 3 \
  --priority 0 \
  --program Rscript /shared/bioinformatics/software/R/generate_gwas_plots.R \
  --in $outDir/processing/${filePrefix}.exclude_singletons+p2df_eq_0.$df.table \
  --in_chromosomes autosomal_nonPAR \
  --in_header \
  --out $outDir/processing/${filePrefix}.exclude_singletons+p2df_eq_0.ylim_9.$df \
  --col_id VARIANT_ID \
  --col_chromosome CHR \
  --col_position POSITION \
  --col_p P \
  --col_variant_type TYPE \
  --generate_snp_indel_manhattan_plot \
  --manhattan_odd_chr_color red \
  --manhattan_even_chr_color blue \
  --manhattan_points_cex 1.5 \
  --manhattan_cex_lab 1.5 \
  --manhattan_cex_axis 1.5 \
  --generate_snp_indel_qq_plot \
  --qq_lines \
  --qq_points_bg black \
  --qq_lambda \
  --manhattan_ylim 9

mv $outDir/processing/*.png \
 $outDir/final

### END Generate plots ###


### START Filter by p-value  < 0.001 ###
if [ $df == "1df" ]; then
    pcol=7
else
    pcol=11
fi

outFile=$outDir/processing/${filePrefix}.$df.p_lte_0.001
head -n 1 $outDir/final/${filePrefix}.chr1.$df > $outFile
for (( chr=${minchr}; chr<=${maxchr}; chr++ )); do
  inFile=$outDir/final/${filePrefix}.chr$chr.$df
  echo Processing $inFile
  tail -n +2 $inFile |
    perl -slane 'if ($F[$pcol] <= 0.001) { print; }' -- -pcol=$pcol >> $outFile
done


# Filter by p-value < 0.001 (exclude_singletons)
outFile=$outDir/processing/${filePrefix}.exclude_singletons+p2df_eq_0.$df.p_lte_0.001
head -n 1 $outDir/final/${filePrefix}.chr1.exclude_singletons+p2df_eq_0.$df > $outFile
for (( chr=${minchr}; chr<=${maxchr}; chr++ )); do
  inFile=$outDir/final/${filePrefix}.chr$chr.exclude_singletons+p2df_eq_0.$df
  echo Processing $inFile
  tail -n +2 $inFile |
    perl -slane 'if ($F[$pcol] <= 0.001) { print; }' -- -pcol=$pcol >> $outFile
done

# end p-value filtering


# Sort p-value filtered file (Rscript)
R
df <- "2df"
outDir <- "/shared/jmarks/hiv/meta/results/020"
study <- "uhs1-4_aa+uhs1-4_ea"
filePrefix <- paste("hiv_acquisition_", df, "_meta_analysis_", study, sep="")

# excluding singletons
inData2 <- read.table(paste(sep="", outDir, "/processing/", filePrefix, ".exclude_singletons+p2df_eq_0.", df, ".p_lte_0.001"), header = TRUE)
outData2 <- inData2[order(inData2$P.value),]
write.csv(outData2, file=paste(sep="", outDir, "/final/", filePrefix, ".exclude_singletons+p2df_eq_0.", df, ".p_lte_0.001.csv"), row.names = FALSE, quote=FALSE)
### END Filter by p-value ###



### Create FUMA file ###
# Chr 
# Position
# MarkerName
# P-value
# Allele1
# Allele2
# Effect
# StdErr

#MarkerName chr position Allele1 Allele2 Effect StdErr P-value Direction HetISq HetChiSq HetDf HetPVal
base_dir=$outDir/final

in_file() {
echo $1/hiv_acquisition_2df_meta_analysis_uhs1-4_aa+uhs1-4_ea.chr$2.exclude_singletons+p2df_eq_0.2df;
 }
outfile=$base_dir/hiv_acquisition_2df_meta_analysis_uhs1-4_aa+uhs1-4_ea.exclude_singletons+p2df_eq_0.ALL_CHR.FUMA
echo $(in_file $base_dir 1)

# print header to file
awk '{print $1,$2,$3,$4,$5,$6,$7,$8;exit}'\
    $(in_file $base_dir 1) > $outfile
for (( chr=${minchr}; chr<=${maxchr}; chr++ )); do
    echo processing chr$chr
    awk 'NR>=2{print $1,$2,$3,$4,$5,$6,$7,$8}'\
     $(in_file $base_dir $chr) >> $outfile
done

### End FUMA creation ###

## Secondary scripts
These were scripts called in the methods file above.

### merge_metal

In [ ]:
#!/share/apps/R/bin/Rscript

args <- commandArgs(TRUE)

loop = TRUE
hasMetalFile = FALSE
hasProbabelFiles = FALSE
hasProbabelLabels = FALSE
hasChrColumns = FALSE
hasPosColumns = FALSE
hasProbabelMergeBys = FALSE
hasOutFile = FALSE
metalCsv = FALSE
probabelCsv = FALSE
outCsv = FALSE

while (loop) {

    if (args[1] == "--metal") {
        metalFile = args[2]
        hasMetalFile = TRUE
    }
    
    if (args[1] == "--metal_csv") {
        metalCsv = TRUE
    }
    
    if (args[1] == "--probabel") {
        moreProbabelFiles = TRUE
        while (moreProbabelFiles) {
            if (exists("probabelFiles")) {
                probabelFiles = c(probabelFiles, args[2])
            } else {
                probabelFiles = args[2]
                hasProbabelFiles = TRUE
            }
            if ((length(args) > 2) && (substr(args[3],1,2) != "--")) {
                args = args[2:length(args)]
            } else {
                moreProbabelFiles = FALSE
            }
        }
    }
    
    if (args[1] == "--probabel_labels") {
        moreProbabelLabels = TRUE
        while (moreProbabelLabels) {
            if (exists("probabelLabels")) {
                probabelLabels = c(probabelLabels, args[2])
            } else {
                probabelLabels = args[2]
                hasProbabelLabels = TRUE
            }
            if ((length(args) > 2) && (substr(args[3],1,2) != "--")) {
                args = args[2:length(args)]
            } else {
                moreProbabelLabels = FALSE
            }
        }
    }
    
    if (args[1] == "--probabel_csv") {
        probabelCsv = TRUE
    }
    
    if (args[1] == "--chr_columns") {
        moreChrColumns = TRUE
        while (moreChrColumns) {
            if (exists("chrColumns")) {
                chrColumns = c(chrColumns, args[2])
            } else {
                chrColumns = args[2]
                hasChrColumns = TRUE
            }
            if ((length(args) > 2) && (substr(args[3],1,2) != "--")) {
                args = args[2:length(args)]
            } else {
                moreChrColumns = FALSE
            }
        }
    }
    
    if (args[1] == "--pos_columns") {
        morePosColumns = TRUE
        while (morePosColumns) {
            if (exists("posColumns")) {
                posColumns = c(posColumns, args[2])
            } else {
                posColumns = args[2]
                hasPosColumns = TRUE
            }
            if ((length(args) > 2) && (substr(args[3],1,2) != "--")) {
                args = args[2:length(args)]
            } else {
                morePosColumns = FALSE
            }
        }
    }
    
    if (args[1] == "--probabel_merge_bys") {
        moreProbabelMergeBys = TRUE
        while (moreProbabelMergeBys) {
            if (exists("probabelMergeBys")) {
                probabelMergeBys = c(probabelMergeBys, args[2])
            } else {
                probabelMergeBys = args[2]
                hasProbabelMergeBys = TRUE
            }
            if ((length(args) > 2) && (substr(args[3],1,2) != "--")) {
                args = args[2:length(args)]
            } else {
                moreProbabelMergeBys = FALSE
            }
        }
    }
    
    if (args[1] == "--out") {
        outFile = args[2]
        hasOutFile = TRUE
    }
    
    if (args[1] == "--out_csv") {
        outCsv = TRUE
    }
    
    if (length(args) > 1) {
        args = args[2:length(args)]
    } else {
        loop=FALSE
    }
}

if (!hasMetalFile) {
    cat("No METAL file specified\n")
    q(save = "no", status = 1, runLast = FALSE)  # terminate R session if no METAL files included in argument
}
if (!hasProbabelFiles) {
    cat("No Probabel files specified\n")
    q("no", 1, FALSE) 
}
if (!hasProbabelLabels) {
    cat("No Probabel labels specified\n")
    q("no", 1, FALSE)
}
if (!hasChrColumns) {
    cat("No chromosome columns specified\n")
    q("no", 1, FALSE)
}
if (!hasPosColumns) {
    cat("No position columns specified\n")
    q("no", 1, FALSE)
}
if (!hasProbabelMergeBys) {
    cat("No merge by columns specified\n")
    q("no", 1, FALSE)
}
if (!hasOutFile) {
    cat("No output file specified\n")
    q("no", 1, FALSE)
}

probabelFileCount = length(probabelFiles)
if (length(probabelLabels) != probabelFileCount) {
    cat("# of Probabel labels must match # of Probabel files\n")
    q("no", 1, FALSE)
}
if (length(chrColumns) != probabelFileCount) {
    cat("# of chromosome columns must match # of Probabel files\n")
    q("no", 1, FALSE)
}
if (length(posColumns) != probabelFileCount) {
    cat("# of position columns must match # of Probabel files\n")
    q("no", 1, FALSE)
}
if (length(probabelMergeBys) != probabelFileCount) {
    cat("# of merge by columns must match # of Probabel files\n")
    q("no", 1, FALSE)
}

cat("Reading", metalFile, "\n")
if (metalCsv) {
    merged = read.csv(metalFile, header=TRUE)
} else {
    merged = read.table(metalFile, header=TRUE)
}
for (i in 1:probabelFileCount) {
    cat("Merging", probabelFiles[i], "\n")
    if (probabelCsv) {
        probabel = read.csv(probabelFiles[i], header=TRUE)
    } else {
        probabel = read.table(probabelFiles[i], header=TRUE)
    }
    colnames(probabel) = paste(colnames(probabel), probabelLabels[i], sep=".")
    merged = merge(merged, probabel, by.x = "MarkerName", by.y = paste(probabelMergeBys[i], probabelLabels[i], sep="."), all.x = TRUE)
    chrColumn = paste(chrColumns[i], probabelLabels[i], sep=".")
    if (chrColumn %in% colnames(merged)) {
        if ("chr" %in% colnames(merged)) {
            merged[is.na(merged$chr),"chr"] = merged[is.na(merged$chr),chrColumn]
        } else {
            colnames(merged)[colnames(merged)==chrColumn] = "chr"
        }
        merged = merged[,!(colnames(merged) %in% chrColumn)]
    }
    posColumn = paste(posColumns[i], probabelLabels[i], sep=".")
    if (posColumn %in% colnames(merged)) {
        if ("position" %in% colnames(merged)) {
            merged[is.na(merged$position),"position"] = merged[is.na(merged$position), posColumn]
        } else {
            colnames(merged)[colnames(merged)==posColumn] = "position"
        }
        merged = merged[,!(colnames(merged) %in% posColumn)]
    }
}
leadingColumns = c("MarkerName","chr","position")
merged = cbind(merged[,colnames(merged) %in% leadingColumns], merged[,!(colnames(merged) %in% leadingColumns)])
merged = merged[order(merged$P.value),]
cat("Writing", outFile, "\n")
if (outCsv) {
    write.csv(merged, file = outFile, row.names = FALSE)
} else {
    write.table(merged, file = outFile, row.names = FALSE, quote = FALSE)
}
cat("Done\n")


### qsub_metal.sh

Note the change at the top for metalScript. We are using a different METAL version. We had to add a patch in order to perform 2df (interaction) calculations. See email below:

```css
Dana asked me to send you the methods for running the joint 2df meta-analysis. I have attached 3 files to this email:
1) METAL-i.docx - instructions for installing the needed software and running the meta-analysis.
2) metal-2010-02-08-patch2.tar.gz - a patch for METAL (the meta-analysis software) that adds the joint 2df functionality.
2) generic-metal-2010-02-08.tar.gz - the version of METAL that you will need to apply the patch to (the patch will not work on newer versions of METAL).


```

In [ ]:
#!/bin/sh

#metalScript="#!/shared/bioinformatics/software/third_party/metal_v3.25.2011/bin/metal\n\n"
metalScript="#!/shared/bioinformatics/software/third_party/generic-metal/bin/metal\n\n"
outFile=""
finalOutFile=""
scriptPrefix=""
mem=3.8
priority=0

while [ "$1" != "" ]; 
do
    if [ "$1" == "--MARKER" ] || [ "$1" == "--EFFECT" ] || [ "$1" == "--PVALUE" ] || [ "$1" == "--WEIGHT" ] || [ "$1" == "--GENOMICCONTROL" ] || [ "$1" == "--PROCESS" ] || [ "$1" == "--SCHEME" ] || [ "$1" == "--STDERR" ] || [ "$1" == "--INTEFFECTLABEL" ] || [ "$1" == "--INTSTDERRLABEL" ] || [ "$1" == "--INTCOVLABEL" ]; then
        metalScript=$metalScript$(echo $1 | sed 's/--//')" "$2"\n"
        shift
    elif [ "$1" == "--ALLELE" ] || [ "$1" == "--OUTFILE" ]; then
        metalScript=$metalScript$(echo $1 | sed 's/--//')" "$2" "$3"\n"
        if [ "$1" == "--OUTFILE" ]; then
            outFile=$2"1"$3
            finalOutFile=$2$3
        fi
        shift
        shift
    elif [ "$1" == "--ANALYZE" ]; then
        if [ "$2" == "HETEROGENEITY" ]; then
            metalScript=$metalScript"ANALYZE HETEROGENEITY\n"
            shift
        else
            metalScript=$metalScript"ANALYZE\n"
        fi
    else
        case $1 in
            --script_prefix )            shift
                                        scriptPrefix=$1
                                        ;;
            --mem )                        shift
                                        mem=$1
                                        ;;
            --priority )                shift
                                        priority=$1
                                        ;;
        esac
    fi
    shift
done

metalScript=$metalScript"QUIT\n"
fileMetalScript=$scriptPrefix".metal.sh"
echo -e $metalScript > $fileMetalScript
chmod 775 $fileMetalScript

fileQsubScript=$scriptPrefix".metal.qsub.sh"
fileQsubScriptLog=$scriptPrefix".metal.qsub.log"
fileQsubScriptError=$scriptPrefix".metal.qsub.error"
echo \#$ -S /bin/bash > $fileQsubScript
echo \#$ -v LD_LIBRARY_PATH >> $fileQsubScript
echo \#  >> $fileQsubScript
echo \# >> $fileQsubScript
echo \#$ -l h_rt=600:00:00  >> $fileQsubScript
echo \# >> $fileQsubScript
echo \# >> $fileQsubScript
echo \# Job name  >> $fileQsubScript
echo \#$ -N METAL >> $fileQsubScript
echo \# >> $fileQsubScript
echo \# Use current working directory >> $fileQsubScript
echo \#$ -cwd >> $fileQsubScript
echo \# >> $fileQsubScript
echo \# Join stdout and stderr >> $fileQsubScript
echo \#$ -j y >> $fileQsubScript
echo \# >> $fileQsubScript
echo \# pe request for MPICH2. Set your number of processors here.  >> $fileQsubScript
echo \# SMP stands for "Shared Memory Jobs" >> $fileQsubScript
echo \#$ -pe smp 1 >> $fileQsubScript
echo \# >> $fileQsubScript
echo \# Run job through bash shell >> $fileQsubScript
echo \#$ -S /bin/bash >> $fileQsubScript
echo \# >> $fileQsubScript
echo \# The following is for reporting only. It is not really needed >> $fileQsubScript
echo \# to run the job. It will show up in your output file. >> $fileQsubScript
echo echo \"Got \$NSLOTS processors.\" >> $fileQsubScript
echo $fileMetalScript >> $fileQsubScript
echo mv $outFile $finalOutFile >> $fileQsubScript
echo mv $outFile.info $finalOutFile.info >> $fileQsubScript

chmod 775 $fileQsubScript

vmem=`echo "$mem + 2" | bc`

qsub -q all.q -l mem_free=${mem}G,h_vmem=${vmem}G,h_rt=24:00:00 -p $priority -o $fileQsubScriptLog -e $fileQsubScriptError $fileQsubScript
